In [16]:
from bs4 import BeautifulSoup as bsoup
import urllib.robotparser
import requests
import pandas as pd
import numpy as np
from pprint import pprint
from selenium import webdriver
import time

In [17]:
# Import GeckoDriverManager module.
from webdriver_manager.firefox import GeckoDriverManager
# Install the GeckoDriverManager to run FireFox web browser.
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())



====== WebDriver manager ======
Current firefox version is 1 cannot be loaded. 
Get LATEST driver version for 1 cannot be loaded. 
Driver [C:\Users\Sam\.wdm\drivers\geckodriver\win64\v0.29.1\geckodriver.exe] found in cache


In [6]:
# formats to scrape, based on Bandcamp URL options
FORMATS = ['all',
           'digital',
           'vinyl',
           'cd',
           'cassette']

# search categories
#   top = best-selling
#   new = new arrivals
#   rec = artist-recommended
SEARCH_CATEGORIES = ['top','new','rec']

# scrape dataframe
scrape_data = pd.DataFrame(columns=['Release Title', 
                                    'Artist Name', 
                                    'Artist Location',
                                    'Release Date',
                                    'Release URL',
                                    'Release Genre',
                                    'Release Sub-Genre',
                                    'Track Information',
                                    'Fan Popularity Score',
                                    'Album Tags'])

# dictionary entry for track listing
track_info_entry = {
    'Track Title' : '',
    'Track Lyrics' : '',
    'Track Number' : '',
    'Track Duration': ''
}

In [7]:
# scrape bandcamp release page function
def scrape_release_page(URL):
    
    pass

In [34]:
# scrape search page function
def scrape_search_page(URL):

    driver.get(URL)
    
    # this is just to ensure that the page is loaded
    time.sleep(1) 
    html = driver.page_source

    # create bsoup object
    soup = bsoup(str(html), 'lxml')    

    # find album link tags on this page
    link_tags = soup.find_all('a', {'class':'item-title'})

    # extract URLs from link tags
    release_URLS = []
    for tag in link_tags:
        release_URLS.append(tag['href'])
    
    for release_URL in release_URLS:
        scrape_release_page(release_URL)
    
    #if response.status_code == 200:
    #    print(URL)
    #    soup = bsoup(response.text, 'lxml')
    #    pprint(soup)

In [9]:
###################### INPUTS ################################

# list of genres to scrape
genres = ['all','rock']

# scrape for subgenres within genres?
scrape_subgenres = True

# dictionary of subgenres to scrape, if scrape_subgenres = True
SUBGENRES = {
                'rock': ['all','indie','prog-rock','post-rock','rock-roll','psychedelic-rock'],
                'metal': [],
                'alternative': [],
                'hip-hop-rap': [],
                'experimental': [],
                'punk': [],
                'folk': [],
                'pop': [],
                'acoustic': [],
                'funk': [],
                'country': [],
                'blues': []
            }

# list of search categories to scrape
search_categories = ['top']

# list of locations to scrape
# location = 0 returns search results for all locations
locations = [0] 

# list of formats to scrape
formats = ['all']

# number of pages to scrape
pages = 10 

###########################################################


In [35]:
# web scraper loop

for genre in genres:
    for category in search_categories:
        for location in locations:
            for formatt in formats:
                for page in range(pages):
                    
                    scrape_URL = 'https://bandcamp.com/?g=' + genre + '&s=' + category + '&p=' + str(page) + '&gn=' + str(location) + '&f=' + formatt
                    scrape_search_page(scrape_URL)
                    break
                    
                    if scrape_subgenres and genre in SUBGENRES:
                        for subgenre in SUBGENRES[genre]:
                            subgenre_scrape_URL = scrape_URL + '&t=' + subgenre
                            scrape_search_page(subgenre_scrape_URL)           

['https://cherubs.bandcamp.com/album/slo-blo-4-frnz-sxy?from=discover-top',
 'https://newretrowave.bandcamp.com/album/love-kills-the-demon?from=discover-top',
 'https://cuedotrecords.bandcamp.com/album/ash-dome?from=discover-top',
 'https://tossportal.bandcamp.com/album/still-slipping-vol-1?from=discover-top',
 'https://boniver.bandcamp.com/album/bon-iver-10th-anniversary-edition?from=discover-top',
 'https://creepingdeathtx.bandcamp.com/album/the-edge-of-existence?from=discover-top',
 'https://orangemilkrecords.bandcamp.com/album/wlfgrl-vinyl?from=discover-top',
 'https://pakt-moonjune.bandcamp.com/album/pakt-live-in-kennett-square?from=discover-top']
['https://cherubs.bandcamp.com/album/slo-blo-4-frnz-sxy?from=discover-top',
 'https://wednesdayband.bandcamp.com/album/twin-plagues-2?from=discover-top',
 'https://quicksandnyc.bandcamp.com/album/distant-populations?from=discover-top',
 'https://selbalamir.bandcamp.com/album/swell?from=discover-top',
 'https://tysegall.bandcamp.com/album